ValueError: len(left_on) must equal the number of levels in the index of "right"
https://stackoverflow.com/questions/28228781/why-does-pandas-inner-join-give-valueerror-lenleft-on-must-equal-the-number-o

In [1]:
import pandas as pd
import numpy as np
pd.options.display.notebook_repr_html = False  # jupyter notebook上での出力形式を制御するために書いています。無くても動きます。

In [2]:
# 動作環境の確認
print(pd.__version__)
print(np.__version__)

1.1.2
1.19.1


## mergeを使うべきところでjoinを使ったのが原因。

* onに複数列を指定する

pandasのコミッターであるsinhrksさんの記事がとても秀逸なので、DataFrameを連結・結合する処理で困ったらここを見よう。  
http://sinhrks.hatenablog.com/entry/2015/01/28/073327

その後、公式ドキュメントにも追加されました。最新の仕様についてはこちらを見たほうが良い。  
本記事の例は公式ドキュメントのものを一部変更している。  
https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [34]:
df_left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

df_right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

print(df_left)
print("~~~~~~")
print(df_right)

  key1 key2   A   B
0   K0   K0  A0  B0
1   K0   K1  A1  B1
2   K1   K0  A2  B2
3   K2   K1  A3  B3
~~~~~~
  key1 key2   C   D
0   K0   K0  C0  D0
1   K1   K0  C1  D1
2   K1   K0  C2  D2
3   K2   K0  C3  D3


In [26]:
result = pd.merge(df_left,df_right, on=['key1', 'key2'])
result

  key1 key2   A   B   C   D
0   K0   K0  A0  B0  C0  D0
1   K1   K0  A2  B2  C1  D1
2   K1   K0  A2  B2  C2  D2

さらに、DataFrameに対してmerge関数を適用することもできる。結果は全く同じになる。

In [22]:
result = df_left.merge(df_right, on=['key1', 'key2'])
result

  key1 key2   A   B   C   D
0   K0   K0  A0  B0  C0  D0
1   K1   K0  A2  B2  C1  D1
2   K1   K0  A2  B2  C2  D2

上記において、間違えてmergeではなくjoinを使ってしまった。

In [23]:
result = df_left.join(df_right, on=['key1', 'key2'])

ValueError: len(left_on) must equal the number of levels in the index of "right"

一応注記しておくと、エラー文の中の"right"はDataFrameの変数名とは無関係である。  
join中で右側に指定したDataFrame、という意味であろう。

上記の話は、「joinではなくmergeを使えば良い」で終わりである。  
では、他の場合でもmergeではなくjoinにしてしまったら、どういうエラーが出るのだろうか?試してみよう。

### mergeとjoinの違い。

と、そのまえにmergeとjoinの違いを見ておこう。

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html
→
https://github.com/pandas-dev/pandas/blob/v1.1.2/pandas/core/frame.py#L7749-L7871
を見ると
joinはその内部でmerge関数を実行しているのが分かる。


### joinにはleft_on, right_onがない

joinにはleft_on, right_onというパラメータがないので、
「そんなパラメータは指定できません」というエラーを返す。これは分かりやすいね。

In [3]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html

df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [1, 2, 3, 5]})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [5, 6, 7, 8]})

In [4]:
df1

  lkey  value
0  foo      1
1  bar      2
2  baz      3
3  foo      5

In [5]:
df2

  rkey  value
0  foo      5
1  bar      6
2  baz      7
3  foo      8

In [6]:
df1.merge(df2, left_on='lkey', right_on='rkey')

  lkey  value_x rkey  value_y
0  foo        1  foo        5
1  foo        1  foo        8
2  foo        5  foo        5
3  foo        5  foo        8
4  bar        2  bar        6
5  baz        3  baz        7

In [7]:
df1.join(df2, left_on='lkey', right_on='rkey')

TypeError: join() got an unexpected keyword argument 'left_on'

### これは何なんだ?  
……？？？


In [8]:
df1 = pd.DataFrame({'key': ['foo', 'bar', 'baz', 'foo'],
                    'value_df1': [1, 2, 3, 5]})
df2 = pd.DataFrame({'key': ['foo', 'bar', 'baz', 'foo'],
                    'value_df2': [5, 6, 7, 8]})

In [9]:
df1.merge(df2, on='key')

   key  value_df1  value_df2
0  foo          1          5
1  foo          1          8
2  foo          5          5
3  foo          5          8
4  bar          2          6
5  baz          3          7

In [10]:
df1.join(df2, on='key')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

## joinで起きる?

### 普通のjoin


In [ ]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                    index=['K0', 'K1', 'K2'])
left

In [ ]:
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                     index=['K0', 'K2', 'K3'])
right

In [ ]:
left.join(right)

### rightのindexを重複

In [ ]:
right_dup_1 = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                     index=['K0', 'K2', 'K2'])
right_dup_1

In [ ]:
left.join(right_dup_1)

In [ ]:
right_dup_2 = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': [111, 222, 333]},
                     index=['K0', 'K2', 'K2'])
right_dup_2

In [ ]:
left.join(right_dup_1)

### leftのindexを重複

In [ ]:
left_dup_1 = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                    index=['K0', 'K0', 'K2'])
left_dup_1

In [ ]:
left_dup_1.join(right)

### leftとrightのindexを重複

In [ ]:
left_dup_1.join(right_dup_1)

In [ ]:
left.join(right, how='inner')

In [ ]:
left_dup_1.join(right, how='inner')